## Importation des bliblothèques

In [115]:
import sqlite3
import pandas as pd

## Connection à la base de données

In [116]:
connect = sqlite3.connect("../../Databases/raw-database.db")
cursor = connect.cursor()

## Importation des données freeze frames

In [117]:
req = cursor.execute(f"SELECT * FROM freeze_frames")
res = req.fetchall()
desc = req.description
freeze_frames = pd.DataFrame(res)
freeze_frames.columns = [i[0] for i in desc]

In [118]:
# Aperçu des données
freeze_frames.head(3)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object
0,4820,2025-01-10 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN
1,4834,2025-01-10 00:04:30.400000,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,1,1020089,away team,159756.0
2,4854,2025-01-10 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN


## Importation des données events

In [119]:
# On récupère les event_id SB pour pouvoir importer uniquement ceux qui nous intéressent depuis la table events
event_id_freeze_frames = freeze_frames.event_id.unique().tolist()

In [120]:
req = cursor.execute(f"SELECT event_id, pass_cross, pass_type FROM events WHERE event_id IN ({', '.join('?' * len(event_id_freeze_frames))})",
                     event_id_freeze_frames)
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [121]:
# Aperçu des données
events.head(3)

,event_id,pass_cross,pass_type
0,2e85ad3e-59c4-479a-a478-4c98174f686b,NaN,None
1,872df8f3-a750-4b61-8890-61a35e96b4a4,1.0,None
2,0f842e18-1363-4af3-8faa-6c9cce55b118,NaN,None


## Jointure des events avec les freeze frames

In [122]:
freeze_frames_events = pd.merge(freeze_frames, events, how = "left", on = "event_id")
freeze_frames_events.sort_values(by = ["match_id_SKC", "timestamp"], inplace = True)

In [123]:
# Aperçu des données
freeze_frames_events.head(3)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,pass_cross,pass_type
0,4820,2025-01-10 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,NaN,None
1,4834,2025-01-10 00:04:30.400000,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,1,1020089,away team,159756.0,1.0,None
2,4854,2025-01-10 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN,NaN,None


## Analyse des centres

In [124]:
# Création d'un dataframe ne comprenant que les centres du dataframe initial
centres = freeze_frames_events[freeze_frames_events.pass_cross == 1]

In [125]:
# On regarde si il y'a des valeurs centres qui sont présents plusieurs fois
# On utilise l'identifiant ["frame", "match_id_SKC"] qui permet d'identifier, de manière unique, chaque frame des données
centres[["frame", "match_id_SKC"]].duplicated().any()

np.False_

On souhaite maintenant identifier chaque centre et chaque évènement précédant un centre par un centre_id.  
Chaque couple (event précédant le centre, centre) sera alors identifié par un unique centre_id.  

De plus, on part du principe qu'on dispose bien de chaque évènement précédant les centres.  
En effet, étant donné que nous ne nous sommes pas chargé de fusionner les évènements précédant et suivant les centres aux évènements correspondant aux centres, nous ne pouvons pas gérer ces erreurs.

Pour gérer les cas ou l'évènement suivant un centre (centre1) est également un centre (centre2), nous partons du principe que ces centres ne sont pas dupliqués mais plutôt que ces centres se suivent en terme de frame.  
En effet, nous n'aurons pas 2 couples (event avant centre1, centre1, centre2) et (centre1, centre2, event après centre2) mais plutot un couple (event avant centre1, centre1, centre2, event après centre2)

In [126]:
# Nous partons de ce principe car nous n'avons pas de valeur dupliquée pour la colonne event_id :
print(freeze_frames_events.event_id.is_unique)

True


Enfin, il est nécessaire des supprimer les events correspondants à des coups de pied arretés car ce n'est pas cohérent d'étudier les évènements précédant des CPA.

In [127]:
# Liste des CPA définis par Stats Bomb
CPA = ["Corner", "Free Kick", "Goal Kick", "Kick Off"]

# Supression des CPA des centres
centres = centres[~centres.pass_type.isin(CPA)]

In [128]:
# identification des évènements précédant les centres
index_events_avant_centre = centres.index - 1

In [129]:
# Création d'un dataframe ne comprenant que les actions précédents les centres
events_avant_centre = freeze_frames_events.loc[index_events_avant_centre]

In [130]:
# On regarde si il y'a des évènements précédents des centres qui correspondent à des centres
events_avant_centre[events_avant_centre.pass_cross == 1]

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,pass_cross,pass_type


In [131]:
# Autre moyen de vérifier
print(len(index_events_avant_centre.intersection(centres.index)))

0


Il n'y en a pas

In [132]:
# Attribution des centre_id à chaque events précédant les centres et à chaque centre
events_avant_centre["centre_id"] = range(len(events_avant_centre))
centres["centre_id"] = range(len(events_avant_centre))

In [133]:
# On attribue aux events précédant les centres l'équipe qui a effectué le centre (suivant ces events)
# En effet, il est possible que l'équipe en possession du ballon lors de l'évènement précédant un centre ne soit pas la même
# que celle qui a effectué le centre en question
# De ce fait, il est nécessaire de remplacer l'équipe actuellement attribuée aux events précédant les centres par les équipes ayant effectué 
# les centres
events_avant_centre.drop("group", axis = 1, inplace = True)
events_avant_centre = pd.merge(events_avant_centre, centres[["group", "centre_id"]], on = "centre_id")

In [134]:
req = cursor.execute(f"SELECT * FROM freeze_frames_data")
res = req.fetchall()
desc = req.description
freeze_frames_data = pd.DataFrame(res)
freeze_frames_data.columns = [i[0] for i in desc]

In [142]:
freeze_frames_data = pd.merge(events_avant_centre[["match_id_SKC", "frame", "centre_id", "group"]], freeze_frames_data, on = ["match_id_SKC", "frame"])

In [154]:
print("Nombre de centres pour lesquels nous avons pas d'information sur la frame de l'event précédent le centre dans les données freeze frames data :", len(events_avant_centre[~events_avant_centre.centre_id.isin(freeze_frames_data.centre_id)]))
print("Nombre total de frames correspondant à des events précédant un centre :", len(events_avant_centre))

Nombre de centres pour lesquels nous avons pas d'information sur la frame de l'event précédent le centre dans les données freeze frames data : 38
Nombre total de frames correspondant à des events précédant un centre : 6181


In [193]:
freeze_frames_data.head(3)

,match_id_SKC,frame,centre_id,group,track_id,trackable_object,is_visible,x,y,vx,vy,speed_norm,accel_norm,z
0,1020089,4820,0,away team,12235.0,12235.0,1,45.43,-0.97,2.594984,0.087762,2.596468,-0.145596,NaN
1,1020089,4820,0,away team,12327.0,12327.0,1,32.26,-2.78,5.386337,-0.666400,5.427404,0.198751,NaN
2,1020089,4820,0,away team,2340.0,2340.0,1,33.55,-16.15,5.579671,-0.015822,5.579693,0.400358,NaN


In [ ]:
Nous avons terminé la première partie 

SELECT *
FROM (SELECT team_id_SB, team_id_SKC AS away_team_id_SKC FROM matching_teams) S JOIN (
SELECT S1.team_id_SKC AS home_team_id_SKC, S3.match_id_SKC, S2.away_team_id, S3.match_id_SB
FROM matching_teams S1 JOIN (SB_matches S2 JOIN matching_matches S3 
ON S2.match_id = S3.match_id_SB)
ON S1.team_id_SB = S2.home_team_id)
ON S.team_id_SB = away_team_id
;

In [168]:
req = cursor.execute(f"SELECT * FROM matching_matches")
res = req.fetchall()
desc = req.description
matching_matches = pd.DataFrame(res)
matching_matches.columns = [i[0] for i in desc]

In [175]:
matching_matches.head(2)

,match_id_SB,match_id_SKC
0,3894366.0,1547880
1,3894367.0,1547881


In [173]:
req = cursor.execute(f"SELECT * FROM matching_teams")
res = req.fetchall()
desc = req.description
matching_teams = pd.DataFrame(res)
matching_teams.columns = [i[0] for i in desc]

In [176]:
matching_teams.head(2)

,team_id_SB,team_id_SKC
0,168,85
1,144,66


In [172]:
req = cursor.execute(f"SELECT match_id AS match_id_SB, home_team_id AS home_team_id_SB, away_team_id AS away_team_id_SB FROM SB_matches")
res = req.fetchall()
desc = req.description
SB_matches = pd.DataFrame(res)
SB_matches.columns = [i[0] for i in desc]

In [178]:
SB_matches.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB
0,3894290,136,139
1,3894367,165,152


In [181]:
merge1 = pd.merge(SB_matches, matching_matches, on = "match_id_SB")

In [191]:
merge2 = pd.merge(merge1, matching_teams, left_on = "home_team_id_SB", right_on = "team_id_SB")
merge2 = merge2.drop("team_id_SB", axis = 1).rename({"team_id_SKC" : "home_team_id_SKC"}, axis = 1)
merge2.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB,match_id_SKC,home_team_id_SKC
0,3894290,136,139,1434956,70
1,3894367,165,152,1547881,72


In [192]:
merge3 = pd.merge(merge2, matching_teams, left_on = "away_team_id_SB", right_on = "team_id_SB")
merge3 = merge3.drop("team_id_SB", axis = 1).rename({"team_id_SKC" : "away_team_id_SKC"}, axis = 1)
merge3.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB,match_id_SKC,home_team_id_SKC,away_team_id_SKC
0,3894290,136,139,1434956,70,86
1,3894367,165,152,1547881,72,328
